This code bases on https://github.com/zaidalyafeai/Notebooks/blob/master/Sketcher.ipynb by Zaid Alyafeai. The model is trained on full Quick, Draw! dataset, instead of just a subset of it.

In [1]:
import requests

# GitHub raw file URL
repo_url = "https://raw.githubusercontent.com/googlecreativelab/quickdraw-dataset/master/categories.txt"

# Send a GET request to download the file
response = requests.get(repo_url)

# Check if the request was successful
if response.status_code == 200:
    # Get the content of the file
    content = response.text
    
    # Save the content to a local file
    with open('categories.txt', 'w') as file:
        file.write(content)
    
    print('File saved successfully.')
else:
    print('Error:', response.status_code)


File saved successfully.


In [2]:
f = open("categories.txt","r")
classes = f.readlines()
f.close()

In [3]:
classes = [c.replace('\n','').replace(' ','_') for c in classes]

In [9]:
!mkdir data

In [10]:
import urllib.request
def download():
  
  base = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/'
  for c in classes:
    cls_url = c.replace('_', '%20')
    path = base+cls_url+'.npy'
    urllib.request.urlretrieve(path, 'data/'+c+'.npy')

In [11]:
download() 

https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/aircraft%20carrier.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/airplane.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/alarm%20clock.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/ambulance.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/angel.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/animal%20migration.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/ant.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/anvil.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/apple.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/arm.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/asparagus.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/axe.npy
https://storage.googleapis.com/quickd

In [4]:
import os
import glob
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras 
import tensorflow as tf

In [7]:
def load_data(root, vfold_ratio=0.2, max_items_per_class= 4000):
    all_files = glob.glob(os.path.join(root, '*.npy'))

    # Initialize variables
    x = np.empty([0, 784])
    y = np.empty([0])
    class_names = []

    # Load each data file
    for idx, file in enumerate(all_files):
        data = np.load(file)
        data = data[0: max_items_per_class, :]
        labels = np.full(data.shape[0], idx)

        x = np.concatenate((x, data), axis=0)
        y = np.append(y, labels)

        class_name, ext = os.path.splitext(os.path.basename(file))
        class_names.append(class_name)
        print(idx, ': finished: ' + class_name)

    data = None
    labels = None

    # Randomize the dataset
    permutation = np.random.permutation(y.shape[0])
    x = x[permutation, :]
    y = y[permutation]

    # Separate into training and testing
    vfold_size = int(x.shape[0] / 100 * (vfold_ratio * 100))

    x_test = x[0:vfold_size, :]
    y_test = y[0:vfold_size]

    x_train = x[vfold_size:x.shape[0], :]
    y_train = y[vfold_size:y.shape[0]]
    return x_train, y_train, x_test, y_test, class_names


In [8]:
x_train, y_train, x_test, y_test, class_names = load_data('data')

0 : finished: aircraft_carrier
1 : finished: airplane
2 : finished: alarm_clock
3 : finished: ambulance
4 : finished: angel
5 : finished: animal_migration
6 : finished: ant
7 : finished: anvil
8 : finished: apple
9 : finished: arm
10 : finished: asparagus
11 : finished: axe
12 : finished: backpack
13 : finished: banana
14 : finished: bandage
15 : finished: barn
16 : finished: baseball
17 : finished: baseball_bat
18 : finished: basket
19 : finished: basketball
20 : finished: bat
21 : finished: bathtub
22 : finished: beach
23 : finished: bear
24 : finished: beard
25 : finished: bed
26 : finished: bee
27 : finished: belt
28 : finished: bench
29 : finished: bicycle
30 : finished: binoculars
31 : finished: bird
32 : finished: birthday_cake
33 : finished: blackberry
34 : finished: blueberry
35 : finished: book
36 : finished: boomerang
37 : finished: bottlecap
38 : finished: bowtie
39 : finished: bracelet
40 : finished: brain
41 : finished: bread
42 : finished: bridge
43 : finished: broccoli


In [9]:
num_classes = len(class_names)
image_size = 28

In [13]:
# Reshape and normalize
x_train = x_train.reshape(x_train.shape[0], image_size, image_size, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], image_size, image_size, 1).astype('float32')

x_train /= 255.0
x_test /= 255.0

# Convert class vectors to class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [18]:
# Define model
model = keras.Sequential()
model.add(layers.Convolution2D(16, (3, 3),
                        padding='same',
                        input_shape=x_train.shape[1:], activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(32, (3, 3), padding='same', activation= 'relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(64, (3, 3), padding='same', activation= 'relu'))
model.add(layers.MaxPooling2D(pool_size =(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='tanh'))
model.add(layers.Dense(num_classes, activation='softmax')) 
# Train model
adam = tf.keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['top_k_categorical_accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 32)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 64)          18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 3, 3, 64)         0

In [19]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
epochs = 30
batch_size = 256

In [20]:
model.fit(x = x_train, y = y_train, 
          validation_split=0.1, 
          verbose=2, 
          epochs = epochs,
          batch_size = batch_size,
          callbacks=[early_stopping]
          # steps_per_epoch=640,
          # validation_data=(x_test_basic_reshape.astype(np.float32), y_test_basic_reshape.astype(np.float32)),
          # validation_freq=20
          )

Epoch 1/30
3882/3882 - 222s - loss: 2.7499 - top_k_categorical_accuracy: 0.6641 - val_loss: 2.1650 - val_top_k_categorical_accuracy: 0.7673 - 222s/epoch - 57ms/step
Epoch 2/30
3882/3882 - 227s - loss: 2.0094 - top_k_categorical_accuracy: 0.7911 - val_loss: 1.9468 - val_top_k_categorical_accuracy: 0.7996 - 227s/epoch - 58ms/step
Epoch 3/30
3882/3882 - 232s - loss: 1.8337 - top_k_categorical_accuracy: 0.8167 - val_loss: 1.8250 - val_top_k_categorical_accuracy: 0.8179 - 232s/epoch - 60ms/step
Epoch 4/30
3882/3882 - 229s - loss: 1.7410 - top_k_categorical_accuracy: 0.8296 - val_loss: 1.7645 - val_top_k_categorical_accuracy: 0.8264 - 229s/epoch - 59ms/step
Epoch 5/30
3882/3882 - 222s - loss: 1.6814 - top_k_categorical_accuracy: 0.8378 - val_loss: 1.7083 - val_top_k_categorical_accuracy: 0.8351 - 222s/epoch - 57ms/step
Epoch 6/30
3882/3882 - 228s - loss: 1.6393 - top_k_categorical_accuracy: 0.8437 - val_loss: 1.6896 - val_top_k_categorical_accuracy: 0.8362 - 228s/epoch - 59ms/step
Epoch 7/30

In [21]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test accuarcy: {:0.2f}%'.format(score[1] * 100))

Test accuarcy: 85.01%


In [23]:
model.save('keras.h5')